In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fine-tuning-llama3/__results__.html
/kaggle/input/fine-tuning-llama3/__notebook__.ipynb
/kaggle/input/fine-tuning-llama3/__output__.json
/kaggle/input/fine-tuning-llama3/custom.css
/kaggle/input/fine-tuning-llama3/wandb/run-20240724_013420-7d4ryncc/run-7d4ryncc.wandb
/kaggle/input/fine-tuning-llama3/wandb/run-20240724_013420-7d4ryncc/logs/debug.log
/kaggle/input/fine-tuning-llama3/wandb/run-20240724_013420-7d4ryncc/logs/debug-internal.log
/kaggle/input/fine-tuning-llama3/wandb/run-20240724_013420-7d4ryncc/files/wandb-summary.json
/kaggle/input/fine-tuning-llama3/wandb/run-20240724_013420-7d4ryncc/files/conda-environment.yaml
/kaggle/input/fine-tuning-llama3/wandb/run-20240724_013420-7d4ryncc/files/config.yaml
/kaggle/input/fine-tuning-llama3/wandb/run-20240724_013420-7d4ryncc/files/output.log
/kaggle/input/fine-tuning-llama3/wandb/run-20240724_013420-7d4ryncc/files/requirements.txt
/kaggle/input/fine-tuning-llama3/wandb/run-20240724_013420-7d4ryncc/files/wandb-metadata.js

The %%capture command is a magic command in Jupyter notebooks that suppresses the output of the cell.
• %pip install -U bitsandbytes upgrades and installs the 'bitsandbytes' Python package.
• %pip install -U transformers upgrades and installs the 'transformers' Python package.
• %pip install -U accelerate upgrades and installs the 'accelerate' Python package.
• %pip install -U peft upgrades and installs the 'peft' Python package.
• %pip install -U trl upgrades and installs the 'trl' Python package.
• The -U flag in the pip install command stands for --upgrade, which upgrades the package if it's already installed.

In [2]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U accelerate
%pip install -U peft
%pip install -U trl

In [3]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("huggi_fac")
login(token = hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
base_model = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"
new_model = "/kaggle/input/fine-tuning-llama3/llama-3-8b-chat-doctor/"

Merging the base model with the adapter

We’ll first load the tokenizer and base model using the transformers library. Then, we’ll set up the chat format using the trl library. Finally, we’ll load and merge the adapter to the base model using the PEFT library.


The merge_and_unload() function will help us merge the adapter weights with the base model and use it as a standalone model.

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel
import torch
from trl import setup_chat_format

2024-08-02 02:25:04.105815: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-02 02:25:04.105916: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-02 02:25:04.248906: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model)

base_model_reload = AutoModelForCausalLM.from_pretrained(
        base_model,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
# Merge adapter with base model
base_model_reload, tokenizer = setup_chat_format(base_model_reload, tokenizer)
model = PeftModel.from_pretrained(base_model_reload, new_model)

In [8]:
model = model.merge_and_unload()


In [9]:
messages = [{"role": "user", "content": "Hello doctor, I have bad acne. How do I get rid of it?"}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

outputs = pipe(prompt, max_new_tokens=120, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

<|im_start|>user
Hello doctor, I have bad acne. How do I get rid of it?<|im_end|>
<|im_start|>assistant
Hi, Welcome to icliniq.com. For your query, I will suggest the following: 1. Cleanse your face with a gentle cleanser in the morning and night. 2. Use a spot treatment for the acne. 3. Avoid popping or squeezing the acne as it can lead to scars. 4. Avoid touching your face as much as possible. 5. Consider using an oral antibiotic such as Doxycycline or Minocycline for 3 to 4 months. 6. Consider using a topical retinoid gel or cream for 3


In [10]:
model.save_pretrained("llama-3-8b-chat-doctor")
tokenizer.save_pretrained("llama-3-8b-chat-doctor")

('llama-3-8b-chat-doctor/tokenizer_config.json',
 'llama-3-8b-chat-doctor/special_tokens_map.json',
 'llama-3-8b-chat-doctor/tokenizer.json')

In [11]:
model.push_to_hub("llama-3-8b-chat-doctor", use_temp_dir=False)
tokenizer.push_to_hub("llama-3-8b-chat-doctor", use_temp_dir=False)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/Deepakiitm/llama-3-8b-chat-doctor/commit/0eaa2a78ffd7585a692c5049e79e91a81e37fafb', commit_message='Upload tokenizer', commit_description='', oid='0eaa2a78ffd7585a692c5049e79e91a81e37fafb', pr_url=None, pr_revision=None, pr_num=None)